In [1]:
import requests
import json
import csv
import pandas as pd
from config import omdb_key
from pprint import pprint

In [2]:
#import csv
musical_csv = "../Resources/Musical_List.csv"

In [3]:
#convert csv to dataframe
musical_movie_sales = pd.read_csv(musical_csv)
musical_movie_sales1 = musical_movie_sales.copy()
musical_movie_sales1.head()

,Rank,Released,Movie,Worldwide Box Office,Domestic Box Office,International Box Office
0,1,2013,Frozen,"$1,267,856,756","$400,953,009","$866,903,747"
1,2,2017,Beauty and the Beast,"$1,255,080,655","$504,014,165","$751,066,490"
2,3,2017,Coco,"$798,237,597","$210,460,015","$587,777,582"
3,4,2016,Moana,"$636,155,760","$248,757,044","$387,398,716"
4,5,2008,Mamma Mia!,"$603,604,074","$144,130,063","$459,474,011"


In [4]:
#set up omdb api connection
url = "http://www.omdbapi.com/?t="
api_key_param = "&apikey=" + omdb_key


In [5]:
#pull data from api
response = requests.get(url + "Frozen" + api_key_param)
data = response.json()
pprint(data)

{'Actors': 'Kristen Bell, Idina Menzel, Jonathan Groff, Josh Gad',
 'Awards': 'Won 2 Oscars. Another 79 wins & 60 nominations.',
 'BoxOffice': '$400,953,009',
 'Country': 'USA, Norway',
 'DVD': '15 Nov 2015',
 'Director': 'Chris Buck, Jennifer Lee',
 'Genre': 'Animation, Adventure, Comedy, Family, Fantasy, Musical',
 'Language': 'English, Norwegian',
 'Metascore': '75',
 'Plot': 'When the newly crowned Queen Elsa accidentally uses her power to '
         'turn things into ice to curse her home in infinite winter, her '
         'sister Anna teams up with a mountain man, his playful reindeer, and '
         'a snowman to change the weather condition.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTQ1MjQwMTE5OF5BMl5BanBnXkFtZTgwNjk3MTcyMDE@._V1_SX300.jpg',
 'Production': 'Walt Disney Animation',
 'Rated': 'PG',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.4/10'},
             {'Source': 'Rotten Tomatoes', 'Value': '90%'},
             {'Source': 'Metacritic', 'V

In [5]:
#Add data to dataframe using loop
movie_data_df = []
movie_info = {}

for index,row in musical_movie_sales1.iterrows():
    try:
        movie_name = row['Movie']
        response = requests.get(url + movie_name + api_key_param)
        data = response.json()
        actors = data.get("Actors")
        movie_info['Movie']= movie_name.copy()
        movie_info['Actors'] = actors.copy()
        print(actors)
        movie_info['Rating'] = data.get('Metascore')
        print(data.get('Metascore'))
        movie_info['Director'] = data.get('Director')
        print(data.get('Director'))
        movie_data_df.append(movie_info.copy())

    except:
        print(f'No Movie {movie_name} found')
        print(data)
       

No Movie Frozen found
No Movie Beauty and the Beast found
No Movie Coco found
No Movie Moana found
No Movie Mamma Mia! found
No Movie Tangled found
No Movie La La Land found
No Movie Beauty and the Beast found
No Movie Les Misérables found
No Movie The Greatest Showman found
No Movie Mamma Mia: Here We Go Again! found
No Movie Grease found
No Movie Mary Poppins Returns found
No Movie Chicago found
No Movie The Sound of Music found
No Movie High School Musical 3: Senior Year found
No Movie Cinderella found
No Movie Into the Woods found
No Movie Hairspray found
No Movie Snow White and the Seven Dwarfs found
No Movie Moulin Rouge found
No Movie Dreamgirls found
No Movie The Phantom of the Opera found
No Movie Sweeney Todd: The Demon Barber of Fleet Street found
No Movie Evita found
No Movie The Jungle Book found
No Movie Annie found
No Movie Anastasia found
No Movie Secret Superstar found
No Movie Staying Alive found
No Movie Mary Poppins found
No Movie Burlesque found
No Movie Fantasia f